Now, I attempt a melody generator over the form of Herbie Hancock's "Maiden Voyage." The A section is just "A-7, C-7." I'll start this up by improvising over A minor pentatonic in triplets.

In [66]:
# Ways to improvise over pentatonic:

# random walk (50% up or down)
# ascending/descending two-notes
# ascending/descending three-notes

Let's map each note of the guitar to one number. Mapping notes to their respective MIDI numbers, let's start with low E (E2, 82.41Hz) as '28' and go all the way to a high C on the first string (C6, 1047Hz) as '72'.

In [2]:
# our set of all possible notes
guitar_notes = range(28,73)

In [3]:
# now make two pitch sets for corresponding pentatonic scales
A_min = [28,31,33,36,38]
C_min = [29,31,34,36,39]

# increase range of pitches to several octaves; up to a super high D on guitar
for i in range(1,4):
    for k in range(0,5):
        A_min.append(A_min[k] + i*12)
        C_min.append(C_min[k] + i*12)


print A_min,C_min

[28, 31, 33, 36, 38, 40, 43, 45, 48, 50, 52, 55, 57, 60, 62, 64, 67, 69, 72, 74] [29, 31, 34, 36, 39, 41, 43, 46, 48, 51, 53, 55, 58, 60, 63, 65, 67, 70, 72, 75]


In [8]:
# note to self:
# difference between accessing an attribute, and calling a function
# hey.you vs hey.you()

In [9]:
# Now, let's try to write our function

In [72]:
# module for random_walk through pitch_set
def random_walk(melody,length,index,pitch_set):
    for i in range(0,length):
        flip = np.random.uniform()
        # first check to see if we're at the absolute range of our pitch set
        if index == 0:
            index += 1
            melody.append(pitch_set[index])
            continue
        if index == (len(pitch_set)-1):
            index -= 1
            melody.append(pitch_set[index])
            continue
        
        # otherwise, move up or down the scale with a coin flip
        if flip > .5:
            index += 1
        else:
            index -= 1
        melody.append(pitch_set[index])
    # I want it to return the melody with added on notes and the index so we can bookmark our place
    return melody, index

In [ ]:
# module for two_note pattern down
def two_notes_down(melody,length,index,pitch_set):
    for i in range(0,length):
        while i > 1:
            i -= 2
        if i == 0:
            index += 1
        if i == 1:
            index -= 2
        
        # have to set up a filter if we get to lower end of instrument

In [143]:
# module to change key to nearest notes
def change_key_nn(melody,index,pitch_set1,pitch_set2):
    # if we're on a common note between sets, just return a step of random walk
    if pitch_set1[index] == pitch_set2[index]:
        melody, index = random_walk(melody,1,index,pitch_set1)
        return melody, index
    
    flip = np.random.uniform()
    # if index in first pitch set is higher and we win flip, we move up; otherwise, we move down in pitch and
    # index still says the same; also, vice versa
    if pitch_set1[index] > pitch_set2[index] and flip >.5:
        index +=1
    if pitch_set1[index] < pitch_set2[index] and flip <.5:
        index -=1
        
    #sloppy fix to if we go out of bounds
    if index < 0:
        index = 0
    if index >= len(A_min):
        index = len(A_min) - 1
    
    melody.append(pitch_set2[index])
    return melody, index
    

In [160]:
#global index

import numpy as np
def melody_writer():
    melody = []
    start = np.random.uniform()
    # we'll randomly start at indices 5,6,7
    if start < float(1)/3:
        index = 5
    elif start < float(2)/3:
        index = 6
    else:
        index = 7
    
    melody.append(A_min[index])
    
    melody, index = random_walk(melody,11,index,A_min)
    melody, index = random_walk(melody,12,index,A_min)
    melody, index = change_key_nn(melody,index,A_min,C_min)
    melody, index = random_walk(melody,20,index,C_min)


    
    '''
    for i in range(0,47):
        flip = np.random.uniform()
        # first check to see if we're at the absolute range of our pitch set
        if index == 0:
            index += 1
            melody.append(A_min[index])
            continue
        if index == (len(A_min)-1):
            index -= 1
            melody.append(A_min[index])
            continue
        
        # otherwise, move up or down the scale with a coin flip
        if flip > .5:
            index += 1
        else:
            index -= 1
        melody.append(A_min[index])
    
    '''
    print index
    return melody

melody = melody_writer()

9


In [161]:
print index

48


In [162]:
print melody
print '\n',len(melody)

[43, 40, 38, 36, 33, 36, 33, 31, 28, 31, 33, 36, 38, 36, 38, 40, 43, 45, 43, 45, 48, 45, 43, 40, 41, 43, 41, 39, 36, 39, 36, 39, 41, 39, 41, 43, 46, 48, 51, 53, 51, 53, 51, 48, 51]

45


In [163]:
from midiutil.MidiFile import MIDIFile

# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
#volume = 100

beats = []
for i in range(0,48):
    beats.append(.3333333333*i)
index = 0

for note in melody:
    mod = index
    while mod >=3:
        mod -= 3
    if mod == 0:
        volume = 150
    else:
        volume = 90
    
    pitch = note          
    time = beats[index]      # start on beat 0
    duration = 1         # 1 beat long
    mf.addNote(track, channel, pitch, time, duration, volume)
    index += 1

with open("triplet.mid", 'wb') as outf:
    mf.writeFile(outf)

In [67]:
'''
    # now we move on to the remaining three triads
    # find the nearest two notes
    remaining_sets = [A,Gb,Eb]
    for key in remaining_sets:
        triad = list(key)
        # I find the note closest to our previous note
        # remove the current note so we always move to a different note
        # when we change keys
        if melody[-1] in triad:
            triad.remove(melody[-1])
        closest = min(triad, key=lambda x:abs(x-melody[-1]))
        #print triad
        #print closest
        melody.append(closest)
        
        for i in range(0,2):
            index = key.index(melody[-1])
            flip = np.random.uniform()
            if index == 0:
                index += 1
            if index == len(key) - 1:
                index -= 1
            if flip > .5:
                index += 1
            else:
                index -= 1
            melody.append(key[index])
    
    melody_notes = []
    # now we convert the numbers into notes with octave numbers for display
    notes = ['C','C#','D','Eb','E','F','F#','G','G#','A','Bb','B']
    for note in melody:
        octave = 0
        while note >= 12:
            note -= 12
            octave += 1
        melody_notes.append(str(notes[note]) + str(octave))
    print melody_notes
    return melody
'''

"\n    # now we move on to the remaining three triads\n    # find the nearest two notes\n    remaining_sets = [A,Gb,Eb]\n    for key in remaining_sets:\n        triad = list(key)\n        # I find the note closest to our previous note\n        # remove the current note so we always move to a different note\n        # when we change keys\n        if melody[-1] in triad:\n            triad.remove(melody[-1])\n        closest = min(triad, key=lambda x:abs(x-melody[-1]))\n        #print triad\n        #print closest\n        melody.append(closest)\n        \n        for i in range(0,2):\n            index = key.index(melody[-1])\n            flip = np.random.uniform()\n            if index == 0:\n                index += 1\n            if index == len(key) - 1:\n                index -= 1\n            if flip > .5:\n                index += 1\n            else:\n                index -= 1\n            melody.append(key[index])\n    \n    melody_notes = []\n    # now we convert the numbers 

In [103]:
# NOTE TO SELF: change the function so it doesn't 
# ALWAYS grab the nearest note when switching chords

In [104]:
# Wow, I like this
sample_melody = [48, 52, 55, 57, 52, 49, 46, 49, 54, 55, 51, 55]